In [2]:
import pandas as pd
import numpy as np
import pyspark
import findspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import udf
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.functions import min,max
from pyspark.sql.functions import weekofyear, sum
import pyspark.sql.types as types
import math
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
zf1 = pd.read_csv('/Users/sakshambhatia/Downloads/Data_For_Assignment/4/uber_data_nyc_2016-06_3m_partitioned.csv/part-00000-65909711-74cb-4da9-acbd-3c279273213e.csv.gz')

In [3]:
zf2 = pd.read_csv('/Users/sakshambhatia/Downloads/Data_For_Assignment/4/uber_data_nyc_2016-06_3m_partitioned.csv/part-00001-65909711-74cb-4da9-acbd-3c279273213e.csv.gz')

In [4]:
zf3 = pd.read_csv('/Users/sakshambhatia/Downloads/Data_For_Assignment/4/uber_data_nyc_2016-06_3m_partitioned.csv/part-00002-65909711-74cb-4da9-acbd-3c279273213e.csv.gz')

In [5]:
zf4 = pd.read_csv('/Users/sakshambhatia/Downloads/Data_For_Assignment/4/uber_data_nyc_2016-06_3m_partitioned.csv/part-00003-65909711-74cb-4da9-acbd-3c279273213e.csv.gz')

In [6]:
zf = zf4.append(zf3.append(zf2.append(zf1)))

In [7]:
zf.set_index(zf.index,inplace=True)

In [8]:
#df.to_csv('/Users/sakshambhatia/Documents/fullSetData.csv')

In [9]:
len(zf)

3299999

In [10]:
zf.describe()

,VendorID,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,total_amount
count,3.299999e+06,3.299999e+06,3.299999e+06,3.299999e+06,3.299999e+06,3.299999e+06
mean,1.527929e+00,-7.305836e+01,4.024679e+01,-7.312868e+01,4.028653e+01,1.664624e+01
std,4.992194e-01,8.175607e+00,4.503776e+00,7.858150e+00,4.328784e+00,1.272999e+02
min,1.000000e+00,-1.151684e+02,0.000000e+00,-1.151754e+02,0.000000e+00,-1.953000e+02
25%,1.000000e+00,-7.399184e+01,4.073531e+01,-7.399123e+01,4.073413e+01,8.760000e+00
50%,2.000000e+00,-7.398138e+01,4.075307e+01,-7.397925e+01,4.075380e+01,1.230000e+01
75%,2.000000e+00,-7.396577e+01,4.076868e+01,-7.396158e+01,4.076994e+01,1.836000e+01
max,2.000000e+00,0.000000e+00,5.109830e+01,1.062469e+02,5.031214e+01,1.874423e+05


In [11]:
zf['index']=np.arange(len(zf))

In [12]:
zf.set_index('index')

,VendorID,tpep_pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,total_amount
index,,,,,,,
0,2,2016-06-09T21:06:36.000-07:00,-73.983360,40.760937,-73.977463,40.753979,7.30
1,2,2016-06-09T21:06:36.000-07:00,-73.987106,40.733173,-73.985909,40.766445,17.76
2,1,2016-06-09T21:06:37.000-07:00,-73.984276,40.775009,-73.968498,40.766430,9.30
3,2,2016-06-09T21:06:37.000-07:00,-73.976334,40.785851,-73.971733,40.791969,4.80
4,1,2016-06-09T21:06:38.000-07:00,-73.988029,40.743835,-73.990173,40.731773,8.75
...,...,...,...,...,...,...,...
3299994,2,2016-06-07T11:57:15.000-07:00,-73.968178,40.762550,-73.998390,40.740429,15.80
3299995,1,2016-06-07T11:57:16.000-07:00,-73.980606,40.751019,-73.983719,40.746422,7.30
3299996,1,2016-06-07T11:57:18.000-07:00,-73.965721,40.754444,-73.950455,40.784973,14.75


In [38]:
sc = SparkContext() 
sqlContext = pyspark.SQLContext(sc)
spark = pyspark.sql.SparkSession(sc) 

In [40]:
df0 = spark.read.csv('/Users/sakshambhatia/Downloads/Data_For_Assignment/4/uber_data_nyc_2016-06_3m_partitioned.csv/part-00000-65909711-74cb-4da9-acbd-3c279273213e.csv.gz'
                    ,header='true',inferSchema='true')

In [41]:
df1 = spark.read.csv('/Users/sakshambhatia/Downloads/Data_For_Assignment/4/uber_data_nyc_2016-06_3m_partitioned.csv/part-00001-65909711-74cb-4da9-acbd-3c279273213e.csv.gz'
                    ,header='true',inferSchema='true')

In [42]:
df2 = spark.read.csv('/Users/sakshambhatia/Downloads/Data_For_Assignment/4/uber_data_nyc_2016-06_3m_partitioned.csv/part-00002-65909711-74cb-4da9-acbd-3c279273213e.csv.gz'
                    ,header='true',inferSchema='true')

In [43]:
df3 = spark.read.csv('/Users/sakshambhatia/Downloads/Data_For_Assignment/4/uber_data_nyc_2016-06_3m_partitioned.csv/part-00003-65909711-74cb-4da9-acbd-3c279273213e.csv.gz'
                    ,header='true',inferSchema='true')

In [44]:
df = df0.union(df1.union(df2.union(df3)))

In [19]:
df.count()

3299999

In [45]:
def haversine(lon1,lat1,lon2,lat2): 
    dLon = (lon2-lon1)*math.pi/180.0
    dLat = (lat2-lat1)*math.pi/180.0

    lat1 = (lat1)*math.pi/180.0
    lat2 = (lat2)*math.pi/180.0
  
    a = (pow(math.sin(dLat/2),2)+pow(math.sin(dLon/2),2)*math.cos(lat1)*math.cos(lat2)); 
    rad = 6371
    c = 2*math.asin(math.sqrt(a)) 
    return (rad*c*0.621371) 

In [46]:
haversine_udf = udf(haversine,types.DoubleType())
uber=df.withColumn("distance_miles",haversine_udf("pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude"))

In [22]:
uber.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- distance_miles: double (nullable = true)



In [23]:
uber.describe().show()

+-------+------------------+-------------------+-----------------+-------------------+-----------------+------------------+------------------+
|summary|          VendorID|   pickup_longitude|  pickup_latitude|  dropoff_longitude| dropoff_latitude|      total_amount|    distance_miles|
+-------+------------------+-------------------+-----------------+-------------------+-----------------+------------------+------------------+
|  count|           3299999|            3299999|          3299999|            3299999|          3299999|           3299999|           3299999|
|   mean|1.5279289478572569| -73.05836057064273|40.24678831059782|  -73.1286791673134|40.28652937530498|16.646240335331093|22.167555833907294|
| stddev|0.4992194401191109|  8.175606882912328|4.503776121420567|  7.858149695712172|4.328784275308626|127.29985554432749|327.50130236430664|
|    min|                 1|-115.16835784912108|              0.0|-115.17537689208984|              0.0|            -195.3|               0.0|

In [24]:
g=uber.select('VendorID','pickup_longitude','pickup_latitude',
                   'dropoff_longitude','dropoff_latitude','total_amount','distance_miles')
g.select([count(when(isnan(c), c)).alias(c) for c in g.columns]).show()

+--------+----------------+---------------+-----------------+----------------+------------+--------------+
|VendorID|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|total_amount|distance_miles|
+--------+----------------+---------------+-----------------+----------------+------------+--------------+
|       0|               0|              0|                0|               0|           0|             0|
+--------+----------------+---------------+-----------------+----------------+------------+--------------+



In [25]:
uber.write.csv('/Users/sakshambhatia/Documents/fullUberDataSet.csv')

In [47]:
uber.createOrReplaceTempView('uber')

In [27]:
spark.sql('select min(tpep_pickup_datetime) as dateRange from uber union select max(tpep_pickup_datetime) from uber').show()

+-------------------+
|          dateRange|
+-------------------+
|2016-06-01 12:30:00|
|2016-06-12 21:59:19|
+-------------------+



In [28]:
uber.select(min("tpep_pickup_datetime").alias('start_date')).show()
uber.select(max("tpep_pickup_datetime").alias('end_date')).show()

+-------------------+
|         start_date|
+-------------------+
|2016-06-01 12:30:00|
+-------------------+

+-------------------+
|           end_date|
+-------------------+
|2016-06-12 21:59:19|
+-------------------+



In [29]:
spark.sql("select weekofyear(tpep_pickup_datetime) as week,sum(distance_miles) as total_distance from uber group by weekofyear(tpep_pickup_datetime)").show()

+----+--------------------+
|week|      total_distance|
+----+--------------------+
|  22|3.7738647911456354E7|
|  23| 3.541426417288098E7|
+----+--------------------+



In [30]:
uber.groupBy(weekofyear("tpep_pickup_datetime").alias("date_by_week")).agg(sum("distance_miles")).orderBy("date_by_week").show()

+------------+--------------------+
|date_by_week| sum(distance_miles)|
+------------+--------------------+
|          22|3.7738647911456354E7|
|          23| 3.541426417288098E7|
+------------+--------------------+



In [31]:
spark.sql('select VendorID from uber group by vendorID order by sum(distance_miles) desc limit 1').show()

+--------+
|VendorID|
+--------+
|       1|
+--------+



In [32]:
(uber.select("VendorID","distance_miles").groupBy('VendorID').agg(sum('distance_miles').alias('Sum_distance'))).select("VendorID").orderBy("Sum_distance",ascending=False).show(1)

+--------+
|VendorID|
+--------+
|       1|
+--------+
only showing top 1 row



In [4]:
range_slider = widgets.IntRangeSlider(
    value=[1, 12],
    min=1, max=12., step=1,
    description='day range:',
    
)
range_slider

IntRangeSlider(value=(1, 12), description='day range:', max=12, min=1)

In [34]:
query=("select sum(total_amount) as total_revenue from uber where dayofmonth(tpep_pickup_datetime) between {} and {}"
       .format(range_slider.value[0],range_slider.value[1]))

In [35]:
spark.sql(query).show()

+-------------------+
|      total_revenue|
+-------------------+
|5.493257646035227E7|
+-------------------+



In [50]:
gs=uber.filter((uber["tpep_pickup_datetime"]>="2016-06-10 04:06:40") & (uber["tpep_pickup_datetime"]<="2016-06-12 16:29:14"))
gs.agg(sum('total_amount').alias('total revenue  between 2016-06-10 04:06:40 & 2016-06-12 16:29:14')).show()

+----------------------------------------------------------------+
|total revenue  between 2016-06-10 04:06:40 & 2016-06-12 16:29:14|
+----------------------------------------------------------------+
|                                            1.3916620769971028E7|
+----------------------------------------------------------------+



In [37]:
sc.stop()